# Required information

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

# Load environment variables from .env
aws_account_id = os.getenv('AWS_ACCOUNT_ID')
aws_region = os.getenv('AWS_REGION')

aws_access_key = os.getenv('AWS_ACCESS_KEY')
aws_secret_key = os.getenv('AWS_SECRET_KEY')

s3_bucket_name = os.getenv('S3_BUCKET_NAME')
s3_object_key = os.getenv('S3_OBJECT_KEY')

## Specify the role name and trust policy for the Lambda service
iam_role_name = os.getenv('IAM_ROLE_NAME')

## User-dependent solution variables

In [2]:
# User setup

# Platform
ecr_image_name = "serverless-example"

# Function description
func_description='A test Lambda function'

# API
endpoint = "predict"
method_verb='POST'
stage_name = "test"

## Protocols and performance

In [4]:
# AWS setup
from os import getcwd
from deploy.utils.misc import get_lambda_usage_constraints, get_trust_policy

cwd=getcwd()

## Specify the path to the JSON file
## NOTE: Update with the actual path to your JSON file
trust_policy_file_path = cwd+"/deploy_scripts"
trust_policy = get_trust_policy(trust_policy_file_path)

## Rate limits: Harsh since this will be public facing
## Quota: Low daily limits for the same reason
usage_constraints_file_path = cwd+"/deploy_scripts"
usage_constraints = get_lambda_usage_constraints(usage_constraints_file_path)

## AWS Lambda execution role 
lambda_policy_arn = 'arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'


Loaded JSON:
{'Version': '2012-10-17', 'Statement': [{'Effect': 'Allow', 'Principal': {'Service': 'lambda.amazonaws.com'}, 'Action': 'sts:AssumeRole'}]}
Loaded JSON:
{'rate_limits': {'burstLimit': 10, 'rateLimit': 10.0}, 'quota': {'limit': 100, 'period': 'DAY'}}


# Clients

In [3]:
import boto3

# Set up the IAM client
iam_client = boto3.client('iam', region_name=aws_region)

# Set up the Lambda client
lambda_client = boto3.client('lambda', region_name=aws_region)

# Set up the API Gateway client
gateway_client = boto3.client('apigateway', region_name=aws_region)

# Development steps

1. IAM role image handling;
2. ECR image
3. Lambda function creation;
4. API Gateway


## IAM Role Image Handling

The first step: create a user on IAM with below permissions:

- **IAMUserChangePassword**: a default permission to change password 
- **IAMFullAccess**: Allows IAM management
- **AmazonEC2ContainerRegistryFullAccess**: Allows uploading image to ECR
- **AWSLambda_FullAccess**: Allows access to specific Lambda function given a role 
- **AmazonAPIGatewayAdministrator**: Allows access to specific API Gateway handling 

## ECR image

- Log in on ECR account;
- Delete existent ECR image;
- Create ECR image;
- Build docker image;
- Tag docker image;
- Push docker image to ECR.

## Lambda function creation

- Create function;
- Test lambda function.

## API Gateway

# Deployment pipeline

In [3]:
from os import getcwd, getenv
from dotenv import load_dotenv

from deploy.core import deploy_solution
from deploy.utils.misc import get_lambda_usage_constraints, get_trust_policy

# Load environment variables from .env
load_dotenv()

# Load environment variables from .env
aws_account_id = getenv('AWS_ACCOUNT_ID')
aws_region = getenv('AWS_REGION')

aws_access_key = getenv('AWS_ACCESS_KEY')
aws_secret_key = getenv('AWS_SECRET_KEY')

s3_bucket_name = getenv('S3_BUCKET_NAME')
s3_object_key = getenv('S3_OBJECT_KEY')

## Specify the role name and trust policy for the Lambda service
iam_role_name = getenv('IAM_ROLE_NAME')

account_info={
    "account_id": aws_account_id,
    "region": aws_region,
    "iam_role":iam_role_name
}

activity_info={
    "image_name": ecr_image_name,
    "lambda_function_description": func_description,
    "endpoint": endpoint,
    "method": method_verb,
    "stage": stage_name,
}

cwd=getcwd()
config_path=cwd+"/deploy"
usage_constraints = get_lambda_usage_constraints(config_path)
trust_policy = get_trust_policy(config_path)


configuration_paths={
    "usage_constraints": usage_constraints,
    "trust_policy": trust_policy
}

deployment_info=deploy_solution(account_info, activity_info, configuration_paths)



Loaded JSON:
{'rate_limits': {'burstLimit': 10, 'rateLimit': 10.0}, 'quota': {'limit': 100, 'period': 'DAY'}}
Loaded JSON:
{'Version': '2012-10-17', 'Statement': [{'Effect': 'Allow', 'Principal': {'Service': 'lambda.amazonaws.com'}, 'Action': 'sts:AssumeRole'}]}
Starting execution of Deployment of ML solution...
Starting execution of Docker image upload on AWS ECR...
Logging in on ECR account...
An error occurred: the JSON object must be str, bytes or bytearray, not NoneType
Creating ECR image...
Building docker image...
Tagging docker image...
Pushing docker image to ECR...
Finished execution of Docker image upload on AWS ECR.
Time taken: 17.70 seconds
Starting execution of Role policy attachment...
Finished execution of Role policy attachment.
Time taken: 1.12 seconds
Starting execution of Lambda Function deployment...
Lambda deployment is still in progress. Waiting...
Lambda deployment is still in progress. Waiting...
Lambda deployment is still in progress. Waiting...
Lambda deploym

In [4]:
from json import dump

# Retrieve information from deployment response
api_key=deployment_info['api_key']
api_url = deployment_info['api_url']

deployment_response = {
    'api_key': api_key,
    'api_url': api_url,
    'method_verb': method_verb,
    # Add other information as needed
}

# Save the deployment information to a JSON file
deployment_info_filename = "deployment_info.json"

with open(deployment_info_filename, 'w') as json_file:
    dump(deployment_response, json_file)

print(f"Deployment information saved to {deployment_info_filename}")

Deployment information saved to deployment_info.json


In [5]:
from json import load

# Save the deployment information to a JSON file
deployment_info_filename = "deployment_info.json"

with open(deployment_info_filename, 'r') as json_file:
    json_payload=load(json_file)

method_verb=json_payload['method_verb']
api_key=json_payload['api_key']
api_url=json_payload['api_url']

print(f"Deployment information loaded from {deployment_info_filename}")

Deployment information loaded from deployment_info.json


In [48]:
import json
import requests

# Prepare the event to pass to the Lambda function
example=[1, 10, 3, 4, 5, 6, 7, 8, 11, 25, 2]

headers = {
    'Content-type': 'application/json', 
    'x-api-key': api_key,
}

resp = requests.post(api_url, headers=headers, json=example)

# Check the response and handle it accordingly
if resp.status_code == 200:
    response_data = resp.json()
    print(response_data)
else:
    print("Request failed with status code:", resp.status_code)
    print(resp.text)



[1, 100, 9, 16, 25, 36, 49, 64, 121, 625, 4]


In [45]:
# WIP

import json
import requests
from about_time import about_time

# Prepare the event to pass to the Lambda function
example_sizes=[1, 10, 100, 1000, 10000, 100000, 100000]
durations=[]

with about_time() as total_t:
    for example_size in example_sizes:
        with about_time() as single_t:
            # Transform into json format
            example=list(range(example_size))

            headers = {
                'Content-type': 'application/json', 
                'x-api-key': api_key,
            }

            resp = requests.post(api_url, headers=headers, json=example)

            # Check the response and handle it accordingly
            if resp.status_code == 200:
                response_data = resp.json()
                print(response_data)
            else:
                print("Request failed with status code:", resp.status_code)
                print(resp.text)
        
        durations.append(single_t.duration_human)


Request failed with status code: 502
{"message": "Internal server error"}
Request failed with status code: 502
{"message": "Internal server error"}
Request failed with status code: 502
{"message": "Internal server error"}
Request failed with status code: 502
{"message": "Internal server error"}
Request failed with status code: 502
{"message": "Internal server error"}
Request failed with status code: 502
{"message": "Internal server error"}
Request failed with status code: 502
{"message": "Internal server error"}


In [47]:
durations

[HumanDuration{ value=0.2298033020051662 } -> 229.8ms,
 HumanDuration{ value=0.2277509999985341 } -> 227.8ms,
 HumanDuration{ value=0.25589482099894667 } -> 255.9ms,
 HumanDuration{ value=0.45181277499796124 } -> 451.8ms,
 HumanDuration{ value=0.36862316699989606 } -> 368.6ms,
 HumanDuration{ value=1.2030376620023162 } -> 1.2s,
 HumanDuration{ value=0.9206874879964744 } -> 920.7ms]